# Quantitative Stock Selection

In [1]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm

## Functions

In [2]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

## Data Import and Wrangling

In [3]:
# IMPORT LATEST DATA 

#data_tmp = pd.read_excel("../R-projects/Investment/Stocks/Data/Borsdata_2020-06-01-2.xlsx")
data_tmp = pd.read_excel("equity_data/Borsdata_2021-02-25.xlsx")
data_tmp = data_tmp[1:]

data_tmp = data_tmp.rename({'Performance' : 'Return 3m','Performance.1' : 'Return 6m',
                            'Performance.2' : 'Return 1y', 'Div. Yield': 'Yield', 'Info' : 'Country',
                            'Info.1' : 'List', 'Info.2' : 'Sector', 'Info.3' : 'Industry', 'Info.4' : 'Tick'}, axis=1)

# FORMAT FOR VIEWING PLEASURE
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

In [4]:
# MAKE NUMBERS FLOATS 

data_tmp['Market Cap'] = data_tmp['Market Cap'].replace(',', '') # remove , as 1000 separator

list = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Yield', 'Market Cap', 'Volatility',
        'F-Score', 'ROC', 'ROE', 'FCF', 'Total Equity ', 
        'Return 3m', 'Return 6m', 'Return 1y']

for i in list: 
    data_tmp[i] = pd.to_numeric(data_tmp[i]) # make every number numeric


data_tmp['FCFROE'] = data_tmp['FCF']/data_tmp['Total Equity '] # calc FCFROE
data_tmp['GPA'] = data_tmp['Gross profit']/data_tmp['Tot. Assets'] 

In [5]:
# REMOVE TOO SMALL COMPANIES AND FINANCIALS 

data_tmp = data_tmp.loc[data_tmp['Market Cap'] > 500]
data_tmp = data_tmp.loc[(data_tmp['List'] != 'First North') & (data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]



data_tmp = data_tmp.loc[(data_tmp['Sector'] != 'Financials')]
data_tmp.index = range(len(data_tmp)) # make new index after removing
data = data_tmp

In [6]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
           'F-Score', 'GPA', 'Assets Turn']
for i in columns: 
    data.loc[data[i].isna() ,i] = data[i].median()

# set nan to min
for i in ['Return 3m', 'Return 6m', 'Return 1y']:
    data.loc[data[i].isna() ,i] = data[i].min()
    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data.loc[data[i] < 0 ,i] = data[i].max()
    
# set nan yield & vol to 0
data.loc[data['Yield'].isna(),'Yield'] = 0
data.loc[data['Volatility'].isna(),'Volatility'] = 0

## Rankings and Sorting

In [7]:
# RANK ON DIFFERENT METRICS 

f_score_cut_off = (pd.DataFrame(data.quantile(0.2, axis=0))).loc['F-Score', 0.2]

data['Momentum'] = (data['Return 3m'] + data['Return 6m'] + data['Return 1y'])/3

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT']:
    data[i +' Rank'] = data[i].rank()
    
# Ranking where higher value is better
for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'Momentum', 'GPA', 'Assets Turn']:
    data[i + ' Rank'] = (-data[i]).rank()
    
# Remove low F-Score from Momentum
data_mom_tmp = data.copy()
data_mom_tmp.loc[data['F-Score'] < f_score_cut_off ,'Momentum Rank'] = data_mom_tmp['Momentum Rank'].max()


# Composite ranks
data['Quality Rank'] = (data['ROE Rank'] + data['ROC Rank'] + data['FCFROE Rank'] + 
                        data['GPA Rank'] + data['Assets Turn Rank']).rank()
data['Value Rank'] = (data['P/E Rank'] + data['P/B Rank'] + data['P/S Rank'] + data['P/FCF Rank'] + 
                      data['EV/EBIT Rank'] + data['Yield Rank']).rank()
 
data['Magic Formula'] = (data['ROC Rank'] + data['EV/EBIT Rank']).rank()
data['Siegfrieds'] = (data['ROC Rank'] + data['P/B Rank']).rank()
data['Triple Sort'] = ((data['Value Rank'] + data['Momentum Rank'] + data['Quality Rank'])).rank()
data['Double Sort'] = (data['Value Rank'] + data['Quality Rank']).rank()

In [8]:
# SORTING 

# number of stocks to choose before momentum screen
trend_screen = 40
double_sort_break_off = 40

value_tmp = data.sort_values(by=['Value Rank'])
quality_tmp = data.sort_values(by=['Quality Rank'])
momentum_tmp = data_mom_tmp.sort_values(by=['Momentum Rank'])


trending_value_tmp = value_tmp[0:trend_screen]
trending_value_tmp = trending_value_tmp.sort_values(by='Momentum Rank')
trending_quality_tmp = quality_tmp[0:trend_screen]
trending_quality_tmp = trending_quality_tmp.sort_values(by='Momentum Rank')
magic_formula_tmp = data.sort_values(by=['Magic Formula'])
siegfrieds_tmp = data.sort_values(by=['Siegfrieds'])
triple_sort_tmp = data.sort_values(by=['Triple Sort'])
double_sort = data.sort_values(by=['Double Sort'])
double_sort_mom_tmp = double_sort[0:double_sort_break_off]
double_sort_mom = double_sort_mom_tmp.sort_values(by=['Momentum Rank'])

In [9]:
# FACTOR STRATEGIES 

compact = ['Company', 'List', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'Tick']

compact_mom = ['Company','Tick', 'List', 'Momentum Rank', 'F-Score' ]

# numbers of stocks to select for diff strategies
stocks = 10
magic_formula_stocks = 10
siegfried_stocks = 10
triple_sort_stocks = 20
double_sort_mom_stocks = 10

## VALUE ## 
value = value_tmp[0:stocks]
Value = value[compact]

## QUALITY ## 
quality = quality_tmp[0:stocks]
Quality = quality[compact]

## MOMENTUM ##
momentum = momentum_tmp[0:stocks]
Momentum = momentum[compact_mom]

## TRENDING VALUE ##
trending_value = trending_value_tmp[0:stocks]
Trending_Value = trending_value[compact]

## TRENDING QUALITY ##
trending_quality = trending_quality_tmp[0:stocks]
Trending_Quality = trending_quality[compact]

## MAGIC FORMULA ##
magic_formula = magic_formula_tmp[0:magic_formula_stocks]
Magic_Formula = magic_formula[compact]

## SIEGFRIEDS ##
siegfrieds = siegfrieds_tmp[0:siegfried_stocks]
Siegfrieds = siegfrieds[compact]

## TRIPLE SORT ##
triple_sort = triple_sort_tmp[0:triple_sort_stocks]
Triple_Sort = triple_sort[compact]

## DOUBLE SORT + Momentum ##
double_sort_mom = double_sort_mom[0:double_sort_mom_stocks]
Double_Sort_Mom = double_sort_mom[compact]

In [10]:
Triple_Sort 

,Company,List,Quality Rank,Value Rank,Momentum Rank,Tick
366,Multiconsult,OB Match,10.00,69.50,36.00,MULTI
121,Kindred,Large Cap,8.00,114.00,25.00,KIND SDB
430,Brdr. A&O Johansen,Small Cap,45.50,23.00,86.00,AOJ P
331,StrongPoint,OB Match,15.00,103.00,38.00,STRO
424,Solar,Mid Cap,39.00,11.00,107.00,SOLAR B
26,Byggmax,Mid Cap,30.00,5.00,137.00,BMAX
16,Bergs Timber,Small Cap,58.00,2.00,114.00,BRG B
341,Flugger,Small Cap,70.00,87.00,26.00,FLUG B
88,Prevas,Small Cap,11.00,20.50,155.00,PREV B
395,Kid,OB Match,38.00,47.00,127.00,KID


In [19]:
Momentum

,Company,Tick,List,Momentum Rank,F-Score
86,Odd Molly,ODD,Small Cap,2.00,5.00
171,Sinch,SINCH,Mid Cap,3.00,6.00
151,G5 Entertainment,G5EN,Mid Cap,5.00,9.00
113,Studsvik,SVIK,Small Cap,6.00,7.00
201,Boozt,BOOZT,Mid Cap,7.00,7.00
211,BHG Group,BHG,Mid Cap,8.00,8.00
78,NetEnt,NET B,Mid Cap,9.00,9.00
143,Xvivo Perfusion,XVIVO,Mid Cap,11.00,5.00
221,Ascelia Pharma,ACE,Small Cap,13.00,5.00
91,Pricer,PRIC B,Small Cap,15.00,5.00
